In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
import pickle
import numpy as np

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
X_test= pickle.load(open("cX_ctest","rb"))
y_test= pickle.load(open("cy_ctest","rb"))

We want to normalise the data. In the grayscale image is min is 0 and max is 255.

In [4]:
X_test=X_test/255.0

In [5]:
model = Sequential()

#3 layers

model.add(Conv2D(96,(3,3),input_shape= X_test.shape[1:],padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(72,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2),padding="same"))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2),padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())
#model.add(Dense(64,Activation="relu"))
model.add(Dense(84, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001)))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001)))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001)))
model.add(Dropout(0.1))

#output 
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy", 
              optimizer='adam', 
              metrics=["accuracy"])


In [6]:
def data_generator():
    for i in range(40):
        X_train=pickle.load(open("cX_ctrain"+str(i),"rb"))/255.0
        y_train=pickle.load(open("cy_ctrain"+str(i),"rb"))
        yield X_train, y_train


In [7]:
model.fit(data_generator(), steps_per_epoch=40, epochs=10,batch_size=32,validation_data=(X_test,y_test))

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[905,96,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/sequential/conv2d_1/Conv2D/Conv2DBackpropInput (defined at C:\Users\96910\AppData\Local\Temp\ipykernel_26596\1005593495.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_1473]

Function call stack:
train_function


In [ ]:
model.save("naive_CNN_color_model.h5")